# 光學字元辨識

![一個傀儡程式正在讀取報紙](./images/ocr.jpg)

一般的電腦視覺挑戰是在影像中偵測和解譯文字。這種處理通常被稱為*光學字元辨識* (OCR)。

## 使用電腦視覺服務讀取影像中的文字

**電腦視覺**認知服務為 OCR 工作提供支援，包括：

- 您可以使用 **OCR** API 來讀取包含多種語言的文字。此 API 可同步使用，且當您需要在影像中偵測和讀取少量文字時，其表現出色。
- 較大文件的最佳**讀取** API。此 API 非同步使用，可用於列印和手寫這兩種文字。

您可以透過建立**電腦視覺**資源或**認知服務**資源來使用此服務。

若您還未這樣做，請在您的 Azure 訂用帳戶中建立**認知服務**資源。

> **備註**：若您已經有認知服務資源，只需在 Azure 入口網站中開啟其 **[快速入門]** 頁面並將其金鑰和端點複製到下面的儲存格。否則，可追隨下面的步驟來建立一個認知服務資源。

1. 在其它瀏覽器索引標籤中，透過 https://portal.azure.com 開啟 Azure 入口網站，並用您的 Microsoft 帳戶登入。

2. 按一下 **[&#65291; 建立資源]** 按鈕，搜尋*認知服務*，並建立包含以下設定的**認知服務**資源：
    - **訂用帳戶**： *您的 Azure 訂用帳戶*。
    - **資源群組**： *選取或建立具有唯一名稱的資源群組*。
    - **區域**： *選擇任一可用區域*：
    - **名稱**： *輸入唯一名稱*。
    - **定價層**：S0
    - **我確認已閱讀通知並理解通知內容**：已選取。
3. 等待部署完成。然後前往您的認知服務資源，在 **[概觀]** 頁面上，按一下連結以管理服務金鑰。您將需要端點和金鑰，以便從用戶端應用程式連線到您的認知服務資源。

### 獲取您的認知服務資源之金鑰和端點

若要使用您的認知服務資源，用戶端應用程式需要其端點和驗證金鑰：

1. 在 Azure 入口網站中，您的認知服務資源之 **[金鑰和端點]** 頁面上，複製您的資源之**金鑰 1**並將其貼上到下面的程式碼，取代 **YOUR_COG_KEY**。
2. 複製您的資源之**端點**並將其貼上到下面的程式碼，代替 **YOUR_COG_ENDPOINT**。
3. 透過按一下 **[執行儲存格]** (&#9655;) 按鈕 (位於儲存格左側) 執行下方儲存格中的程式碼。

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_endpoint = 'YOUR_COG_ENDPOINT'

print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

現在您已經設定好金鑰和端點，可以使用電腦視覺服務資源從影像中擷取文字。

我們從 **OCR** API 開始，它使您能夠同步分析影像並讀取其包含的任一文字。在此案列中，您會得到虛構的 Northwind Traders 零售公司的廣告影像，其中包括一些文字。執行下方儲存格以讀取這些文字。 

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import os
%matplotlib inline

# Get a client for the computer vision service
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Read the image file
image_path = os.path.join('data', 'ocr', 'advert.jpg')
image_stream = open(image_path, "rb")

# Use the Computer Vision service to find text in the image
read_results = computervision_client.recognize_printed_text_in_stream(image_stream)

# Process the text line by line
for region in read_results.regions:
    for line in region.lines:

        # Read the words in the line of text
        line_text = ''
        for word in line.words:
            line_text += word.text + ' '
        print(line_text.rstrip())

# Open image to display it.
fig = plt.figure(figsize=(7, 7))
img = Image.open(image_path)
draw = ImageDraw.Draw(img)
plt.axis('off')
plt.imshow(img)

在影像中找到的文字被組織成由區域、行和詞組成的階層式結構，並且程式碼會讀取它們以取出結果。

在結果中，檢視影像上方已讀取的文字。 

## 顯示周框方塊

結果還包括文字行的*周框方塊*座標和在影像中找到的單個詞。執行下方儲存格以查看您從上方取出的廣告影像中文字行的周框方塊。

In [ ]:
# Open image to display it.
fig = plt.figure(figsize=(7, 7))
img = Image.open(image_path)
draw = ImageDraw.Draw(img)

# Process the text line by line
for region in read_results.regions:
    for line in region.lines:

        # Show the position of the line of text
        l,t,w,h = list(map(int, line.bounding_box.split(',')))
        draw.rectangle(((l,t), (l+w, t+h)), outline='magenta', width=5)

        # Read the words in the line of text
        line_text = ''
        for word in line.words:
            line_text += word.text + ' '
        print(line_text.rstrip())

# Show the image with the text locations highlighted
plt.axis('off')
plt.imshow(img)

在結果中，每個文字行的周框方塊都呈矩形顯示在影像上。

## 使用讀取 API

您之前使用的 OCR API 在處理帶有少量文字的影像時表現良好。當您需要讀取大量文字時，例如掃描的文件，您可以使用**讀取** API。這需要多個步驟處理：

1. 將影像提交至電腦視覺服務以便非同步地讀取和分析影像。
2. 等待分析作業完成。
3. 取出分析結果。

執行以下儲存格以使用此處理來讀取給 Northwind Traders 商店管理員的掃描信件中的文字。

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image
import time
import os
%matplotlib inline

# Read the image file
image_path = os.path.join('data', 'ocr', 'letter.jpg')
image_stream = open(image_path, "rb")

# Get a client for the computer vision service
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Submit a request to read printed text in the image and get the operation ID
read_operation = computervision_client.read_in_stream(image_stream,
                                                      raw=True)
operation_location = read_operation.headers["Operation-Location"]
operation_id = operation_location.split("/")[-1]

# Wait for the asynchronous operation to complete
while True:
    read_results = computervision_client.get_read_result(operation_id)
    if read_results.status not in [OperationStatusCodes.running]:
        break
    time.sleep(1)

# If the operation was successfuly, process the text line by line
if read_results.status == OperationStatusCodes.succeeded:
    for result in read_results.analyze_result.read_results:
        for line in result.lines:
            print(line.text)

# Open image and display it.
print('\n')
fig = plt.figure(figsize=(12,12))
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

檢閱結果。此處有完整的信件轉譯，其主要由帶有手寫簽章的印列文字組成。信件的原始影像顯示在 OCR 結果下方 (您可能需要捲動以便參閱該影像)。

## 讀取手寫文字

在上一個範例中，分析影像的要求指定了一種文字辨識模式，此模式使文字*列印*作業最佳化。注意，儘管如此，還是讀取了手寫簽章。

這種讀取手寫文字的能力用處極大。例如，假設您已寫下一條包含購物清單的備註，並想使用您手機上的一款應用程式來讀取此備註並轉譯其包含的文字。

執行下方儲存格以參閱手寫購物清單讀取作業的範例。

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image
import time
import os
%matplotlib inline

# Read the image file
image_path = os.path.join('data', 'ocr', 'note.jpg')
image_stream = open(image_path, "rb")

# Get a client for the computer vision service
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Submit a request to read printed text in the image and get the operation ID
read_operation = computervision_client.read_in_stream(image_stream,
                                                      raw=True)
operation_location = read_operation.headers["Operation-Location"]
operation_id = operation_location.split("/")[-1]

# Wait for the asynchronous operation to complete
while True:
    read_results = computervision_client.get_read_result(operation_id)
    if read_results.status not in [OperationStatusCodes.running]:
        break
    time.sleep(1)

# If the operation was successfuly, process the text line by line
if read_results.status == OperationStatusCodes.succeeded:
    for result in read_results.analyze_result.read_results:
        for line in result.lines:
            print(line.text)

# Open image and display it.
print('\n')
fig = plt.figure(figsize=(12,12))
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

## 更多資訊

有關使用適用於 OCR 的電腦視覺服務之更多資訊，請參閱[電腦視覺文件](https://docs.microsoft.com/zh-tw/azure/cognitive-services/computer-vision/concept-recognizing-text)